# RecommendationSystem

In [44]:
# Function to read valid input for each feature and check for null values
def get_valid_input(attribute_name):
    while True:
        # Read input for each attribute dynamically
        input_value = input(f"Please enter the value for {attribute_name}: ")

        # Check if input is not empty or null
        if input_value.strip():
            return input_value
        else:
            print(f"Input for {attribute_name} is invalid. Please enter a valid value.")

# Define all the features
features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms','lyrics']

# Dictionary to store the valid inputs
feature_inputs = {}

# Loop through each feature and get the valid input
for feature in features:
    feature_inputs[feature] = get_valid_input(feature)

# Output the valid inputs
for feature, value in feature_inputs.items():
    print(f"Valid Input for {feature}: {value}")


Please enter the value for danceability: 0.7
Please enter the value for energy: 0.7
Please enter the value for key: 7
Please enter the value for loudness: 11
Please enter the value for mode: 2
Please enter the value for speechiness: 0.05
Please enter the value for acousticness: 0.0001
Please enter the value for instrumentalness: 0.7
Please enter the value for liveness: 0.4
Please enter the value for valence: 0.4
Please enter the value for tempo: 100
Please enter the value for duration_ms: 53764
Please enter the value for lyrics: I woke up to the sound of the rain, Falling softly on my windowpane. A gentle breeze, the world feels new, I can't help but think of you.
Valid Input for danceability: 0.7
Valid Input for energy: 0.7
Valid Input for key: 7
Valid Input for loudness: 11
Valid Input for mode: 2
Valid Input for speechiness: 0.05
Valid Input for acousticness: 0.0001
Valid Input for instrumentalness: 0.7
Valid Input for liveness: 0.4
Valid Input for valence: 0.4
Valid Input for tempo

In [ ]:
!pip install langdetect

In [45]:
import nltk
from nltk.corpus import words

# Download the NLTK words corpus if not already installed
nltk.download('words')

[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [46]:
# Load the list of English words from the NLTK words corpus
english_words_set = set(words.words())

# Function to check the percentage of words in English
def english_word_percentage(text):
    try:
        if isinstance(text, str) and text.strip():
            # Split the text into words
            words_in_text = text.split()
            # Count words that exist in the English words set
            english_word_count = sum(1 for word in words_in_text if word.lower() in english_words_set)
            # Calculate the percentage of English words
            percentage = english_word_count / len(words_in_text)
            return percentage
        else:
            return 0  # Return 0 if the text is empty or not a string
    except Exception as e:
        print(f"Error processing text: {e}")
        return 0

# Function to validate if the lyrics are valid English
def validate_lyrics_language(lyrics, threshold=0.5):
    percentage = english_word_percentage(lyrics)
    if percentage >= threshold:  # Threshold for detecting English lyrics
        print(f"Valid English Lyrics with {percentage*100:.2f}% English words.")
        return True
    else:
        print(f"Invalid lyrics. Only {percentage*100:.2f}% of the words are in English.")
        return False

# Example usage
input_lyrics = feature_inputs['lyrics']

# Check if the input lyrics are valid English
is_valid = validate_lyrics_language(input_lyrics)


Valid English Lyrics with 75.00% English words.


In [ ]:
!pip install contractions

In [47]:
import contractions
import re
from nltk.corpus import stopwords

# Download the NLTK stopwords (if not already downloaded)
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [48]:
# Function to clean lyrics
def clean_lyrics(lyrics):
    # Step 1: Remove stop words using NLTK
    stop_words = set(stopwords.words('english'))
    words = lyrics.split()
    words = [word for word in words if word.lower() not in stop_words]
    lyrics_no_stopwords = " ".join(words)

    # Step 2: Expand contractions
    lyrics_expanded = contractions.fix(lyrics_no_stopwords)

    # Step 3: Remove special/non-special characters using regular expressions
    lyrics_cleaned = re.sub(r'[^a-zA-Z0-9\s]', '', lyrics_expanded)

    return lyrics_cleaned

# Example valid_lyrics (assuming this is what you got after language detection)
#valid_lyrics = "I'm feeling so sad, I don't know why!"

# Clean the lyrics
basic_cleaning_lyrics = clean_lyrics(input_lyrics)

# Print the cleaned lyrics
print("Cleaned Lyrics:", basic_cleaning_lyrics)


Cleaned Lyrics: woke sound rain Falling softly windowpane gentle breeze world feels new cannot help think you


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer, PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

In [49]:
# Initialize lemmatizer and stemmer
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Function to clean and process lyrics
def process_lyrics(lyrics):
    # Tokenization: Split lyrics into words
    tokens = word_tokenize(lyrics)

    # Lemmatization: Convert tokens to their base forms
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in tokens]

    # Stemming: Convert tokens to their stem forms
    stemmed_tokens = [stemmer.stem(token) for token in tokens]

    return lemmatized_tokens, stemmed_tokens

# Example lyrics input
basic_cleaning_lyrics = "I'm walking down the street and it's raining cats and dogs!"

# Process the cleaned lyrics
lemmatized_tokens, stemmed_tokens = process_lyrics(basic_cleaning_lyrics)

# Output the results
print("Tokenized Lyrics:", word_tokenize(basic_cleaning_lyrics))
print("Lemmatized Tokens:", lemmatized_tokens)
print("Stemmed Tokens:", stemmed_tokens)

Tokenized Lyrics: ['I', "'m", 'walking', 'down', 'the', 'street', 'and', 'it', "'s", 'raining', 'cats', 'and', 'dogs', '!']
Lemmatized Tokens: ['I', "'m", 'walking', 'down', 'the', 'street', 'and', 'it', "'s", 'raining', 'cat', 'and', 'dog', '!']
Stemmed Tokens: ['i', "'m", 'walk', 'down', 'the', 'street', 'and', 'it', "'s", 'rain', 'cat', 'and', 'dog', '!']


### vectorization

In [ ]:
!pip install requests

In [ ]:
import requests
import zipfile
import os

In [50]:
# Define the URL of the GloVe embeddings and the path to save the file
glove_url = "http://nlp.stanford.edu/data/glove.6B.zip"
zip_file_path = "glove.6B.zip"
extract_dir = "glove.6B"

# Download the file from the URL
response = requests.get(glove_url)
with open(zip_file_path, "wb") as file:
    file.write(response.content)

# Extract the downloaded ZIP file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

# Path to the GloVe 50D embeddings
glove_file_path = os.path.join(extract_dir, "glove.6B.50d.txt")

# Verify if the file is extracted
if os.path.exists(glove_file_path):
    print(f"GloVe file successfully extracted to {glove_file_path}")
else:
    print("Failed to extract GloVe file.")

GloVe file successfully extracted to glove.6B/glove.6B.50d.txt


In [51]:
import numpy as np

# Function to load GloVe embeddings (50D version)
def load_glove_model(glove_file):
    glove_model = {}
    with open(glove_file, 'r', encoding='utf-8') as f:
        for line in f:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            glove_model[word] = vector
    return glove_model

# Load the GloVe model (50D vectors)
glove_model = load_glove_model("/content/glove.6B/glove.6B.50d.txt")

In [52]:
def get_glove_vector_for_lyrics(lyrics, glove_model):
    try:
        # Tokenize and convert to lowercase
        tokens = lyrics
        vectors = []

        # Iterate over each token in the lyrics
        for word in tokens:
            # Check if the word exists in the GloVe model
            if word in glove_model:
                vectors.append(glove_model[word])

        # Check if any valid vectors were found
        if vectors:
            # Return the average of all word vectors in the song
            return np.mean(vectors, axis=0)
        else:
            # Return a vector of zeros if no GloVe vectors were found for words in the song
            print(f"No valid GloVe vectors found for the lyrics: {lyrics[:30]}...")
            return np.zeros(50)  # 50D zero vector

    except Exception as e:
        # Handle any error that occurs during the process and print the error message
        print(f"Error processing lyrics: {e}")
        return np.zeros(50)  # Return a zero vector in case of error

In [53]:
# Apply GloVe vectorization to all lyrics
feature_inputs['glove_vector'] = get_glove_vector_for_lyrics(stemmed_tokens, glove_model)
feature_inputs['glove_vector_numeric'] = np.mean(feature_inputs['glove_vector'])

In [54]:
print(feature_inputs)

{'danceability': '0.7', 'energy': '0.7', 'key': '7', 'loudness': '11', 'mode': '2', 'speechiness': '0.05', 'acousticness': '0.0001', 'instrumentalness': '0.7', 'liveness': '0.4', 'valence': '0.4', 'tempo': '100', 'duration_ms': '53764', 'lyrics': "I woke up to the sound of the rain, Falling softly on my windowpane. A gentle breeze, the world feels new, I can't help but think of you.", 'glove_vector': array([ 1.00457855e-01,  5.48056401e-02, -6.67329282e-02, -2.09627151e-01,
        3.15852165e-01, -8.91926438e-02, -5.78443527e-01, -8.31107050e-02,
        1.33464590e-03, -4.24673222e-02, -2.06857190e-01,  1.37035012e-01,
       -3.89777869e-01, -3.87476422e-02,  4.00983602e-01,  1.23331048e-01,
        1.12779297e-01,  1.05782636e-01, -6.21075213e-01, -5.22718191e-01,
       -6.06916472e-02,  3.50971431e-01,  2.84065008e-01,  8.23742896e-02,
        4.78178412e-01, -1.61441064e+00, -5.32594681e-01,  3.84965807e-01,
        4.35715675e-01, -6.14706039e-01,  3.04924250e+00,  2.95632571e-

### Finding the Emoition of the Song

In [ ]:
import joblib

rf_model = joblib.load('/content/rf_model.joblib')

In [57]:
# Emotion mapping dictionary
emotion_mapping = {
    'Neutral': 1,
    'Relaxed': 2,
    'Sad': 3,
    'Angry': 4,
    'Happy': 5,
    'Energetic': 5
}

# Input features (including glove_vector_numeric)
glove_vector_numeric = feature_inputs['glove_vector_numeric']
#features = [0.6, 0.5, 7, -11.157, 2, 0.05, 0.01, 0.7, 0.4, 0.1, 100, 5242, glove_vector_numeric]
feature_inputs['glove_vector_numeric'] = glove_vector_numeric

# Accessing individual features
features = [
    feature_inputs['danceability'],
    feature_inputs['energy'],
    feature_inputs['key'],
    feature_inputs['loudness'],
    feature_inputs['mode'],
    feature_inputs['speechiness'],
    feature_inputs['acousticness'],
    feature_inputs['instrumentalness'],
    feature_inputs['liveness'],
    feature_inputs['valence'],
    feature_inputs['tempo'],
    feature_inputs['duration_ms'],
    feature_inputs['glove_vector_numeric']
]

# Make a prediction using the Random Forest model
y_pred = rf_model.predict([features])

# Map the numeric prediction back to the emotion using emotion_mapping
# Assuming y_pred is a numeric value that corresponds to an emotion in emotion_mapping
predicted_emotion = [emotion for emotion, value in emotion_mapping.items() if value == y_pred[0]]

# Output the predicted emotion
if predicted_emotion:
    print(f"Predicted Emotion: {predicted_emotion[0]}")
else:
    print("Emotion not found in the mapping.")


Predicted Emotion: Neutral


In [73]:
feature_inputs

{'danceability': '0.7',
 'energy': '0.7',
 'key': '7',
 'loudness': '11',
 'mode': '2',
 'speechiness': '0.05',
 'acousticness': '0.0001',
 'instrumentalness': '0.7',
 'liveness': '0.4',
 'valence': '0.4',
 'tempo': '100',
 'duration_ms': '53764',
 'lyrics': "I woke up to the sound of the rain, Falling softly on my windowpane. A gentle breeze, the world feels new, I can't help but think of you.",
 'glove_vector': array([ 1.00457855e-01,  5.48056401e-02, -6.67329282e-02, -2.09627151e-01,
         3.15852165e-01, -8.91926438e-02, -5.78443527e-01, -8.31107050e-02,
         1.33464590e-03, -4.24673222e-02, -2.06857190e-01,  1.37035012e-01,
        -3.89777869e-01, -3.87476422e-02,  4.00983602e-01,  1.23331048e-01,
         1.12779297e-01,  1.05782636e-01, -6.21075213e-01, -5.22718191e-01,
        -6.06916472e-02,  3.50971431e-01,  2.84065008e-01,  8.23742896e-02,
         4.78178412e-01, -1.61441064e+00, -5.32594681e-01,  3.84965807e-01,
         4.35715675e-01, -6.14706039e-01,  3.0492425

In [61]:
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

In [90]:
# Load the dataset
file_path = "/content/final_songs_dataframe.csv"
final_songs_dataframe = pd.read_csv(file_path)

In [91]:
# Check the column names in the dataframe
print(final_songs_dataframe.columns)


Index(['id', 'name', 'album_name', 'artists', 'danceability', 'energy', 'key',
       'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness',
       'liveness', 'valence', 'tempo', 'duration_ms', 'lyrics', 'glove_vector',
       'glove_vector_numeric', 'emotion_category'],
      dtype='object')


In [92]:
final_songs_dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 73394 entries, 0 to 73393
Data columns (total 20 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   id                    73394 non-null  object 
 1   name                  73394 non-null  object 
 2   album_name            73394 non-null  object 
 3   artists               73394 non-null  object 
 4   danceability          73394 non-null  float64
 5   energy                73394 non-null  float64
 6   key                   73394 non-null  object 
 7   loudness              73394 non-null  float64
 8   mode                  73394 non-null  object 
 9   speechiness           73394 non-null  float64
 10  acousticness          73394 non-null  float64
 11  instrumentalness      73394 non-null  float64
 12  liveness              73394 non-null  float64
 13  valence               73394 non-null  float64
 14  tempo                 73394 non-null  float64
 15  duration_ms        

In [93]:
final_songs_dataframe['glove_vector']

,glove_vector
0,[ 0.08974653 0.10316204 0.09011229 -0.287115...
1,[ 0.07517301 0.24146253 0.02544807 -0.359017...
2,[ 2.29687616e-01 6.40893996e-01 2.40405411e-...
3,[ 0.25893155 0.5670942 -0.57082355 0.165255...
4,[ 0.25893155 0.5670942 -0.57082355 0.165255...
...,...
73389,[ 1.0983306e-01 5.7236213e-02 1.3336040e-01 ...
73390,[ 1.2784953e-01 2.0313313e-02 1.1022345e-01 ...
73391,[ 1.94302887e-01 5.86311296e-02 9.58473012e-...
73392,[ 1.52678609e-01 1.18894935e-01 1.22678332e-...


In [110]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Assuming y_pred is the predicted emotion value
emotion_mapping = {
    1: 'Neutral',
    2: 'Relaxed',
    3: 'Sad',
    4: 'Angry',
    5: 'Energetic'
}

# Filter songs by emotion
filtered_songs = final_songs_dataframe[final_songs_dataframe['emotion_category'] == emotion_mapping[y_pred]]

# Function to convert the string representation of the glove_vector to a numpy array
def convert_glove_vector(glove_str):
    glove_str_cleaned = glove_str.replace('[', '').replace(']', '').replace('\n', ' ').strip()  # Remove newlines, brackets, and extra spaces
    glove_vector = np.array(list(map(float, glove_str_cleaned.split())))  # Convert string to list of floats
    return glove_vector

# Convert the 'glove_vector' column strings to numpy arrays using .loc
filtered_songs.loc[:, 'glove_vector'] = filtered_songs['glove_vector'].apply(convert_glove_vector)

# Apply cosine similarity between the feature_inputs['glove_vector'] and each song's glove_vector
filtered_songs['similarity'] = filtered_songs['glove_vector'].apply(
    lambda x: cosine_similarity(feature_inputs['glove_vector'].reshape(1, -1), x.reshape(1, -1))[0][0]
)


<ipython-input-110-cccc41a0557d>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_songs['similarity'] = filtered_songs['glove_vector'].apply(


In [113]:
# Return top 5 songs
recommended_songs = filtered_songs.sort_values(by='similarity', ascending=False).head(5)

# Select only the 'lyrics' and 'similarity' columns
recommended_songs_output = recommended_songs[['lyrics', 'similarity']]

# Set display options for a clean and neat table output
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.width', 1000)        # Set display width to fit more columns
pd.set_option('display.max_colwidth', None) # Prevent truncation of long text in columns

# Display the table (neatly formatted)
display(recommended_songs_output)

,lyrics,similarity
7354,"wake half time, know five minutes think ""What country? city? hotel in?"" five minutes think ""Is wild animal scratching around next bed? large rat raccoon squirrel broke in?"" stumble around room, touching walls, feeling around Feeling around dark feel feel find light switch turn recognize cat there's beautiful woman laying bed you, head clear Wait I'm wrong, thought clear I'm alone There's guitar recording gear Okay, realize I'm hotel room San Francisco stayed girl tour San Francisco long That's really remember, hotel night long engineer's limping around messed knee mosh pit night He's blaming let Come watch McGregor fight apartment ""I already girlfriend over,"" told ""It's last night together Marc Hopkins sessions"" carpet gold, walls yellow, curtains pink carpet gold, walls yellow, curtains pink carpet gold, walls yellow, curtains pink carpet gold, walls yellow, curtains pink I'm back Ireland, I'm totally stoned jet lag permanently clock different day day duration life Lately go bed 7 o'clock morning, wake 4 PM dark New Orleans 5:30 walk across Coliseum Square I'll get iced tea Mojo Magazine sit bit come back window, see Pontchartrain Hotel sign lit like always is, neon green finished book John Connolly, got choked end Like end book, Book Lost Things end Stan Laurel hears Pacific Ocean waves crashing mind, takes past Oliver Hardy audience clapping I'm listening Nels Cline's Lovers I'm watching news mudslides snowstorms east coast bothers like everybody fingerpointing Saying somebody something wrong like news anymore pick remote click I'm falling asleep, I'm dream state I'm back San Francisco, walking around apartment earthquake I'm feeling? guitars falling place floors rising ceiling getting closer ceiling, heart beating faster faster run kitchen, grab food bottled water exit apartment fast look fruit bowl there's unbearably loud hum Coming cable car tracks oranges yellow, lemons green, cable car goes vroom-vroom oranges yellow, lemons green, cable car goes vroom-vroom oranges yellow, lemons green, cable car goes vroom-vroom oranges yellow, lemons green, cable car goes vroom-vroom dream, walked SPCA somebody asked ""Mark, pets?,"" said, ""Yes, do"" says, ""Well, kind pets?"" said, ""Well, take guess"" guesses guesses said ""Well wrong, unique pets"" says, ""How different?"" said, ""Well, let show video them"" said, ""Okay, let watch video them"" show video pets' eyes bulged frightened Like Mia Farrow's eyes Rosemary's Baby fucked Satan kitty cat barks, duck goes quack, puppy dog goes meow kitty cat barks, duck goes quack, puppy dog goes meow kitty cat barks, duck goes quack, puppy dog goes meow kitty cat barks, duck goes quack, puppy dog goes meow friend said, ""Mark, know leaving tonight child look after?"" said, ""Yeah, he's smart kid wants grow actor movie director Steve Railsback favorite actor world, watch movies together son's favorite movies Devil's Rejects, Ed Gein, Helter Skelter"" friend said, ""Well, who's babysitter?"" said, ""Don't worry it, that's business, he's neighbor"" said, ""Look, gotta go, called taxi, son's gonna fine Talk later, alligator"" got babysitter coming, live 666 Post, neighbor is... zoom got babysitter coming, live 666 Post, neighbor is... zoom got babysitter coming, live 666 Post, neighbor is... zoom got babysitter coming, live 666 Post, neighbor is... zoom",0.986262
31781,"Shit got heavy like eyelids highway end got quiet like island left headed sea toothless shark actor playing part can't pretend care like unknown second home we're upside down, love Gravity change heart fell hell fell I've incisive intuition curse anything Always first know everything ends Bullets June nice night knife fight two boats drifting sun shines you, love well-lit like work art I've exiled like enemy Forever dark",0.982193
64402,"Damn go walking find horses man-made lake trees Came back room covered sweat Swiss Waldhaus Hotel Filled application work visa Japan Australia weeks since I've